## This file consist the process of formatting of datasets, the creation of npy files from the formatted dataset, then taking the mfcc of the data in the npy file and then finally using the final data to train the  model and same for all dev set as well except the mfcc and the training part

In [1]:
!pip list


Package              Version             
-------------------- --------------------
absl-py              0.7.1               
astor                0.8.0               
attrs                19.1.0              
audioread            2.1.8               
backcall             0.1.0               
bleach               3.1.0               
blitz                0.3.3               
blockdiag            1.5.4               
bob.extension        3.1.1               
certifi              2019.6.16           
cffi                 1.12.3              
chardet              3.0.4               
Click                7.0                 
click-plugins        1.1.1               
cycler               0.10.0              
decorator            4.4.0               
defusedxml           0.6.0               
entrypoints          0.3                 
funcparserlib        0.3.6               
gast                 0.2.2               
google-pasta         0.1.7               
grpcio               1.23.0       

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [ ]:
import keras 
import numpy as np
# import librosa as lb
import sys
# import pandas as pd
from keras.utils import to_categorical
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import History 
from keras.utils import plot_model,to_categorical
from keras.optimizers import SGD
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler
from iter_window import window 
# import speechpy as sp
# import statistics
from keras import backend as K
from keras.layers import Dense, Activation, Flatten

In [ ]:
filename = "/media/hinton/F8E62A5EE62A1D7E/rohit/spoof/spoof_deep_features/DS_10283_853/protocol/CM_protocol/cm_train.trn"

# open the file for reading
filehandle = open(filename, 'r')
train_protocol = []
while True:
    # read a single line
    line = (filehandle.readline())
    train_protocol.append(line)
    if not line:
        break

# close the pointer to that file
filehandle.close()


In [ ]:
filename_dev = "/media/grace/Drive1/rohit/spoof/spoof_deep_features/DS_10283_853/protocol/CM_protocol/cm_develop.ndx"

# open the file for reading
filehandle_dev = open(filename_dev, 'r')
dev_protocol = []
while True:
    # read a single line
    line = (filehandle_dev.readline())
    dev_protocol.append(line)
    if not line:
        break

# close the pointer to that file
filehandle_dev.close()


In [ ]:
train_protocol1 = []
for i in train_protocol:
    i = i.replace('\n','')
    train_protocol1.append(i)

In [ ]:
dev_protocol1 = []
for i in dev_protocol:
    i = i.replace('\n','')
    dev_protocol1.append(i)

In [ ]:
dev_protocol = dev_protocol1
print(dev_protocol)

In [ ]:
train_protocol = train_protocol1
print(train_protocol)

In [ ]:
train_pd = []
train_label = []
for i in train_protocol:
    j = i.split(' ')[1:2]
    train_pd.append(j)
    train_label.append(i.split(' ')[3:4])
    

In [ ]:
train_pd

In [ ]:
train_pd1 = []
for i in train_pd:
    train_pd1.append(str(i)[2:-2])
train_pd = train_pd1

In [ ]:
len(train_pd)

In [ ]:
dev_pd = []
dev_label = []
for i in dev_protocol:
    dev_pd.append(i.split(' ')[1:2])
    dev_label.append(i.split(' ')[3:4])
    

In [ ]:
dev_pd1 = []
for i in dev_pd:
    dev_pd1.append(str(i)[2:-2])
dev_pd = dev_pd1

In [ ]:
train_pd

In [ ]:
#import names of files in dataset
path = r'/media/hinton/F8E62A5EE62A1D7E/rohit/spoof/spoof_deep_features/DS_10283_853/wav/'
files = []
missing=[]
print(path)
for r, d, f in os.walk(path):
    for file in f:
        if '.wav' in file  :        
            files.append(os.path.join(r, file))
        else:
            missing.append(file)
print(len(files))


In [ ]:
len(missing)

In [ ]:
#get train and dev set
file_name=[]
for i in missing:
    i = i.split('/')[-1].replace('.wav','')
    file_name.append(i)

file_set = set()
train_pd_set = set()
# dev_pd_set = set()
file_set = set(file_name)
train_pd_set = set(train_pd)
# dev_pd_set = set(dev_pd)

train_set = file_set & train_pd_set
# dev_set = file_set & dev_pd_set

In [ ]:
len(train_set)

## This code below ensures that train samples and train_labels are in the same order.

In [ ]:
train_label = []
label_set = []
for k in train_set:
    for i in train_protocol1:
        j = str(i.split(' ')[1:2])[2:-2]
        if j==k:
            label_set.append(j)
            train_label.append(i.split(' ')[3])

In [ ]:
len(train_label)

In [ ]:
len(label_set)

In [ ]:
dev_label = []
for i in dev_protocol:
    j = str(i.split(' ')[1:2])[2:-2]
    if j in dev_set:
        dev_label.append(i.split(' ')[3])

In [ ]:
np.save("train_label.npy",train_label)
# np.save("dev_label.npy",dev_label)

In [ ]:
#To save audio data of trainset
train_audio= []
for count, i in enumerate(train_set):
#     print(i)
    j = i.split('_')[0]
    train_audio.append(lb.load('/media/grace/Drive1/rohit/spoof/spoof_deep_features/DS_10283_853/wav/'+j+'/'+i+'.wav'))
    if count%1000==0 and count==1000:
        print('saving')
        np.save("train"+str(count)+".npy",train_audio[0:count])
    elif count%1000==0 and count >= 1000:
        print('saving')
        np.save("train"+str(count)+".npy",train_audio[count-1000:count])

In [ ]:
#To save audio data of trainset
from os import path
train_audio= []
missing_audio = []
for count, i in enumerate(train_set):
#     print(i)
    j = i.split('_')[0]
    if path.exists('/media/hinton/F8E62A5EE62A1D7E/rohit/spoof/spoof_deep_features/DS_10283_853/wav/'+j+'/'+i+'.wav'):
        train_audio.append(i)
    else:
        missing_audio.append(i)

In [ ]:
train_set_list = list(train_set)


In [ ]:
train_protocol1[100]

In [ ]:
train_audio[1]

In [ ]:
#To save .npy file of audio data of devset
dev_audio= []
for count,i in enumerate(dev_set):
#     print(i)
    j = i.split('_')[0]
    dev_audio.append(lb.load('/media/grace/Drive1/rohit/spoof/spoof_deep_features/DS_10283_853/wav/'+j+'/'+i+'.wav'))
    if count%1000==0 and count==1000:
        print('saving')
        np.save("dev"+str(count)+".npy",dev_audio[0:count])
    elif count%1000==0 and count >= 1000:
        print('saving')
        np.save("dev"+str(count)+".npy",dev_audio[count-1000:count])

In [ ]:
train_audio[1][0]

In [ ]:
train = []
path = r'/media/grace/Drive1/rohit/spoof/spoof_deep_features/weights'
print(path)
for r, d, f in os.walk(path):
    for file in f:
        if '.npy' in file and 'train' in file and '_' not in file:        
            train.append(np.load(os.path.join(r, file), allow_pickle = True))
# print(train)


In [ ]:
dev = []
path = r'/media/grace/Drive1/rohit/spoof/spoof_deep_features/'
print(path)
for r, d, f in os.walk(path):
    for file in f:
        if '.npy' in file and 'dev' in file and '_' not in file:        
            dev.append(np.load(os.path.join(r, file), allow_pickle = True))
print(len(dev))

In [ ]:
# train[16][1]
train_format = []
for i in range(len(train)):
    for j in train[i]:
        train_format.append(j[0]) 

In [ ]:
dev_format = []
for i in range(len(dev)):
    for j in dev[i]:
        dev_format.append(j[0]) 

In [ ]:
train_format

In [ ]:
train_format= np.array(train_format)


In [ ]:
dev_format = np.array(dev_format)

In [ ]:
lb.feature.mfcc(f,22050, n_mfcc=60).shape

In [ ]:
delta = []
delta2 = []
#use librosa to get mfcc features
mfcc_feat_train = np.empty((16000,17880))

for count,f in enumerate(train_format):
    delta=np.array(lb.feature.delta(lb.feature.mfcc(f,22050, n_mfcc=60)))
    delta2=np.array(lb.feature.delta(lb.feature.mfcc(f,22050, n_mfcc=60), order=2))
    value =pad_sequences(sp.processing.cmvnw(np.concatenate([delta,delta2], axis=None).reshape(1,-1)),maxlen=17880,dtype='float32')
    mfcc_feat_train[count] = value
    if count%100==0:
        print(count)

    

In [ ]:
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6]])
np.concatenate((a, b), axis=None)


In [ ]:
mfcc_feat_train.shape

In [ ]:
mfcc_feat_train.shape

In [ ]:
# window_train = []
# for i in mfcc_feat_train:
#     window_train.append(list((window(i[0],7,7,padding=0))))

In [ ]:
mfcc_length =[]
for i in mfcc_feat_train:
    mfcc_length.append(i.shape[1])
print(max(mfcc_length))  
len(mfcc_feat_train)
median = statistics.median(mfcc_length)
print(median)

In [ ]:
#find median of the length of windowed features to padd or chop
# length_windows = []
# for i in range(len(window_train)):
#     length_windows.append(len(window_train[i]))
    
# median = statistics.median(length_windows)


# pad_window = pad_sequences(window_train,maxlen=17880,dtype='float32')

# pad_window[1][17879]

In [ ]:
# data = np.empty((16000,17880),dtype = 'float32')
# for x,i in enumerate(mfcc_feat):
#     for y,j in enumerate(i):
#         out = pad_sequences(j.reshape(1,-1),maxlen=17880,dtype='float32')
#         data[x][y] = out


In [ ]:
data

In [ ]:
np.save("mfcc_trian_padded.npy",mfcc_feat_train,allow_pickle = True)

In [ ]:
import numpy as np

In [ ]:
data.shape

In [ ]:
# delta = []
# delta2 = []
# #use librosa to get mfcc features
# mfcc_feat_dev= []
# for f in dev_format:
# #     mfcc_feat_train.append(np.array((lb.features.delta(lb.feature.mfcc(f[0],22050))).flatten()))
#     delta=np.array(lb.feature.delta(lb.feature.mfcc(f,22050, n_mfcc=60)))
#     delta2=np.array(lb.feature.delta(lb.feature.mfcc(f,22050, n_mfcc=60), order=2))
# #     mfcc_feat_train = np.append(np.array(mfcc_feat_train),np.concatenate([delta,delta2], axis = 1))
#    mfcc_feat_dev.append(np.concatenate([delta,delta2], axis = 1))

# mfcc_length = []
# for i in mfcc_feat_dev:
#     mfcc_length.append(i.shape[1])
# print(max(mfcc_length))    

# data = np.empty((len(mfcc_feat_dev),60,max(mfcc_length)),dtype = 'float32')
# for x,i in enumerate(mfcc_feat_dev):
#     for y,j in enumerate(i):
#         out = pad_sequences(j.reshape(1,-1),maxlen=max(mfcc_length),dtype='float32')
#         data[x][y] = out


# np.save("mffc_dev",data,allow_pickle = True)

In [ ]:
# data_dev = np.load("mffc_dev.npy")

In [ ]:
data_train = np.load("mfcc_trian_padded.npy")

In [ ]:
data_train.shape

In [ ]:
# data_train = data_train.reshape(16000,-1)

In [ ]:
train_labels = np.load("train_label.npy")

train_labels=list(train_labels)

train_labels1 = list()
for i in train_labels:
    if i == b'human':
        train_labels1.append(0)
    else:
        train_labels1.append(1)



In [ ]:
train_labels1.count(1)

In [ ]:
# scalers = {}
# x_train,y_train = np.load("train_data1.npy"), np.load("train_labels1.npy")
# for i in range(x_train.shape[1]):
#     scalers[i] = MinMaxScaler()
#     x_train[:, i, :] = scalers[i].fit_transform(x_train[:, i, :]) 
# x_validation,y_validation = np.load("validation_data1.npy"), np.load("validation_labels1.npy")
# for i in range(x_validation.shape[1]):
#     x_validation[:, i, :] = scalers[i].transform(x_validation[:, i, :]) 
# x_test,y_test = np.load("test_data1.npy"), np.load("test_labels1.npy")
# for i in range(x_test.shape[1]):
#     x_test[:, i, :] = scalers[i].transform(x_test[:, i, :]) 

In [ ]:
#define neural network
output = []
model = Sequential()

model.add(Dense(1000, activation='sigmoid', input_shape = (17880,)))
model.add(Dense(1000, activation='sigmoid'))
model.add(Dense(1000, activation='sigmoid'))
model.add(Dense(1000, activation='sigmoid'))
model.add(Dense(64, activation='linear'))
model.add(Dense(2, activation='softmax'))

# tb = TensorBoard(log_dir="/media/grace/Drive1/rohit/spoof/spoof_deep_features/visuals/logs/{}".format(time()))

# filepath="/media/grace/Drive1/rohit/spoof/spoof_deep_features/weights/weights.best.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]

# model.load_weights("/media/grace/Drive1/rohit/spoof/spoof_deep_features/weights/weights.best.hdf5")

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
layeroutputs = []
# for i in pad_window:
model.fit(data_train, to_categorical(train_labels1[0:16000]), epochs = 10,batch_size = 64)#, callbacks = callbacks_list, validation_data=(x_validation,y_validation))
get_5th_layer_output = K.function([model.layers[0].input],
                                 [model.layers[4].output])
output = get_5th_layer_output([data_train])[0]
    

# y_pred =  model.predict(x_test)
# y_pred = np.round(y_pred[:,1])
# print(metrics.accuracy_score(y_pred,y_test))

In [ ]:
output.shape

In [ ]:
np.save("BNF.npy",output)

In [ ]:
from sklearn.mixture import GaussianMixture as GMM
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from sklearn.utils import shuffle
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# Load Dataset and Labels
## 0 : Human and 1 : spoof

In [ ]:
data_embedding=np.load('BNF.npy')
train_labels = np.load("train_label.npy")

train_labels=list(train_labels)

train_labels1 = list()
for i in train_labels:
    if i == b'human':
        train_labels1.append(0)
    else:
        train_labels1.append(1)
train_labels = train_labels1[0:16000]

# rus = RandomUnderSampler(random_state=0)
# X_resampled, y_resampled = rus.fit_resample(data_embedding, train_labels1[:16000])
# X,Y = shuffle(X_resampled,y_resampled)

In [ ]:
len(train_labels)

In [ ]:
data_embedding.shape

In [ ]:
unique, counts = np.unique(train_labels, return_counts=True)
dict(zip(unique, counts))

## seperate the zeros and the ones.


In [ ]:
train_labels

In [ ]:
j , k = 0,0
# human_samples = np.empty((3674,17880)) # wrong array size as BNF is of size 3674,64
human_samples = np.empty((3674,64))# corrected
human_labels= []
# spoof_samples = np.empty((12326,17880))
spoof_samples = np.empty((12326,64))
spoof_labels = []
for count,i in enumerate(train_labels):
    if i ==0:
#         human_samples=np.append(human_samples,data_embedding[count])# Do not append insert the data at that position
        human_samples[j]=data_embedding[count]
        human_labels.append(train_labels[count])
        j = j+1
    elif i ==1:
#         spoof_samples=np.append(spoof_samples,data_embedding[count])
        spoof_samples[k] = data_embedding[count]
        spoof_labels.append(train_labels[count])
        k = k+1
        
print(len(human_labels))
print(len(spoof_labels))
print((human_samples[3]))
        

In [ ]:
np.save("human_samples.npy",human_samples,allow_pickle=True)
np.save("spoof_samples.npy",spoof_samples,allow_pickle=True)
np.save("human_labels.npy",human_labels)
np.save("spoof_labels.npy",spoof_labels)

# Fit GMM for human samples

In [ ]:
human_samples = np.load("human_samples.npy")
human_labels = np.load("human_labels.npy")
spoof_samples = np.load("spoof_samples.npy")
spoof_labels = np.load("spoof_labels.npy")

In [ ]:
human_samples.shape

In [ ]:
gmm = GMM(n_components=512).fit(human_samples)

In [ ]:
y_predicted = gmm.predict(human_samples)

In [ ]:
plt.scatter(human_samples[:, 0], human_samples[:, 1], c=y_predicted, s=40, cmap='viridis');

In [ ]:
y_predicted.shape

In [ ]:
human_labels.shape

In [ ]:
unique, counts = np.unique(y_predicted, return_counts=True)
dict(zip(unique, counts))

In [ ]:
fpr, tpr, threshold = roc_curve(human_labels, y_predicted, pos_label=1)
eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
thresh = interp1d(fpr, threshold)(eer)

In [ ]:
tpr

In [ ]:
eer

# Fit GMM for spoof samples

In [ ]:
gmm_spoof = GMM(n_components=512).fit(spoof_samples)

In [ ]:
y_predicted_spoof = gmm.predict(spoof_samples)

In [ ]:
unique, counts = np.unique(y_predicted_spoof, return_counts=True)
dict(zip(unique, counts))

In [ ]:
fpr, tpr, threshold = roc_curve(spoof_labels, y_predicted_spoof, pos_label=1)
eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
thresh = interp1d(fpr, threshold)(eer)

In [ ]:
eer

# fit GMM for all samples

In [ ]:
data_embedding=np.load('BNF.npy')
train_labels = np.load("train_label.npy")

In [ ]:
train_labels=list(train_labels)

train_labels1 = list()
for i in train_labels:
    if i == b'human':
        train_labels1.append(0)
    else:
        train_labels1.append(1)
train_labels = train_labels1[0:16000]


In [ ]:
gmm = GMM(n_components=512,covariance_type='full', random_state=0).fit(data_embedding)

In [ ]:
y_predicted = gmm.predict(data_embedding)

In [ ]:
unique, counts = np.unique(y_predicted, return_counts=True)
dict(zip(unique, counts))

In [ ]:
fpr, tpr, threshold = roc_curve(train_labels, y_predicted, pos_label=1)
eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
thresh = interp1d(fpr, threshold)(eer)

In [ ]:
eer